In [ ]:
import traceback
import os
import json
from cpitospin import analyze_cpi_structure, CPIToSPINConverter

# Choose the test
All the test available are in folder 'CPIs'

In [ ]:
process_name = "choice"

## Load File 

In [ ]:
cpi_file_path = f'../CPIs/{process_name}.cpi'

print(f"Loading CPI file: {cpi_file_path}")

try:
	with open(cpi_file_path, 'r') as f:
		cpi_dict = json.load(f)

	print("✓ CPI file loaded successfully!")
	print(f"Root region type: {cpi_dict['type']}")
	print(f"Root region ID: {cpi_dict['id']}")

	# Pretty print the CPI structure
	print("\nCPI Structure:")
	print("=" * 50)
	print(json.dumps(cpi_dict, indent=2))

except FileNotFoundError:
	print(f"❌ File not found: {cpi_file_path}")
	print("Available files in CPIs directory:")
	try:
		for f in os.listdir('../CPIs'):
			if f.endswith('.cpi'):
				print(f"  - {f}")
	except:
		print("  Could not list CPIs directory")
except Exception as e:
	print(f"❌ Error loading CPI file: {e}")
	traceback.print_exc()

# FROM CPI TO SPIN

In [ ]:
print("\nCPI Structure Analysis:")
print("=" * 50)
if 'cpi_dict' in locals():
	analyze_cpi_structure(cpi_dict)

print("Converting CPI to SPIN...")
print("=" * 50)

try:
	converter = CPIToSPINConverter()
	spin_model = converter.convert_cpi_to_spin(cpi_dict)

	print("✓ Conversion successful!")
	print("\nSPIN Model Summary:")
	print("-" * 30)
	spin_model.print_model_summary()

except Exception as e:
	print(f"❌ Conversion failed: {e}")
	traceback.print_exc()

In [ ]:
from cpitospin import create_cpi_visualization, create_spin_visualization

print("\nCreating visualizations...")
print("=" * 50)

try:
	# Create CPI visualization
	cpi_viz = create_cpi_visualization(cpi_dict, "CPI: Loop Example")
	print("✓ CPI visualization created")

	# Create SPIN visualization
	spin_viz = create_spin_visualization(spin_model, "SPIN: Loop Example")
	print("✓ SPIN visualization created")

	# Display visualizations (if in Jupyter)
	try:
		print("\nCPI Structure Visualization:")
		display(cpi_viz)

		print("\nSPIN Model Visualization:")
		display(spin_viz)
	except NameError:
		# Not in Jupyter, save to files instead
		print("Saving visualizations to files...")

		# Save CPI visualization
		cpi_output = process_name + '_cpi'
		cpi_viz.render(cpi_output, cleanup=True)
		print(f"CPI visualization saved to: {cpi_output}.png")

		# Save SPIN visualization
		spin_output = process_name + '_spin'
		spin_viz.render(spin_output, cleanup=True)
		print(f"SPIN visualization saved to: {spin_output}.png")

except Exception as e:
	print(f"❌ Visualization failed: {e}")
	traceback.print_exc()


# FROM SPIN TO PRISM

In [ ]:
print("\nGenerating PRISM model...")
print("=" * 50)

try:
	prism_model = spin_model.generate_prism_model()

	print("✓ PRISM model generated successfully!")

	# Show model statistics
	lines = prism_model.split('\n')
	words = prism_model.split()

	print(f"Model statistics:")
	print(f"  Lines: {len(lines)}")
	print(f"  Words: {len(words)}")
	print(f"  Characters: {len(prism_model)}")

	# Show first 20 lines
	print(f"\nFirst 20 lines of PRISM model:")
	print("-" * 40)
	for i, line in enumerate(lines[:20]):
		print(f"{i + 1:2d}: {line}")

	if len(lines) > 20:
		print(f"... ({len(lines) - 20} more lines)")

except Exception as e:
	print(f"❌ PRISM generation failed: {e}")
	traceback.print_exc()

print("\nSaving PRISM model...")
print("=" * 50)

try:
	output_file =  "../models/" + process_name + '.nm'
	with open(output_file, 'w') as f:
		f.write(prism_model)

	print(f"✓ PRISM model saved to: {output_file}")

	# Verify file was created
	if os.path.exists(output_file):
		file_size = os.path.getsize(output_file)
		print(f"File size: {file_size} bytes")
	else:
		print("❌ Warning: File was not created")

except Exception as e:
	print(f"❌ Save failed: {e}")
	traceback.print_exc()

## RUN PRISM ANALYSIS

In [ ]:
from run_prism import run_prism_analysis
run_prism_analysis(process_name, '/home/emanuelechini/cpi-to-prism-loops/prism-4.8.1-linux64-x86/bin/prism')

In [ ]:
from graphviz import Source
with open(f"../models/{process_name}.dot", 'r', encoding='utf-8') as f: 
    dot_content = f.read() 
    graph = Source(dot_content) # Esporta in SVG graph.render(filename=output_path, format='svg', cleanup=True)
    graph.render(filename=f"../results/{process_name}/{process_name}", format='svg', cleanup=True)

# FROM PRISM TO MDP

Since PRISM models are based on an extended form of MDPs, we now provide the compact version of the MDP generated by PRISM, which corresponds to the equivalent SPIN model. This enhances clarity and facilitates easier comparison between the two representations.

In [ ]:
import pandas as pd 
import pandas as pd

def create_states_mdp(process_name: str = 'test'):
    df = pd.read_csv(f"../results/{process_name}/{process_name}_states.csv")

    df['STAGE'] = df['(STAGE'].apply(lambda x: x.split(':')[-1].split('(')[-1] if isinstance(x, str) else x)
    df.drop(columns=['(STAGE'], inplace=True)

    # Find column ending with ')' and extract second-to-last character
    paren_cols = [col for col in df.columns if col.endswith(')')]
    for col in paren_cols:
        df[col.replace(')', '')] = df[col].apply(lambda x: x[-2] if isinstance(x, str) and len(x) >= 2 else x)
        df.drop(columns=[col], inplace=True)

    # Convert everything to numeric where possible
    df = df.apply(pd.to_numeric)

    # Find columns ending with _update and _state
    update_cols = [col for col in df.columns if col.endswith('_update')]
    state_cols = [col for col in df.columns if col.endswith('_state')]

    # Filter new states
    new_states_df = df[
       # (df['STAGE'] == 0) &
        (df[update_cols] == 0).all(axis=1) &
        (df[state_cols] == 0).all(axis=1)
    ]

    return new_states_df
create_states_mdp(process_name = 'choice')#

In [ ]:
pd.read_csv(f"../results/{process_name}/{process_name}_states.csv")

In [ ]:
pd.read_csv(f"../results/{process_name}/{process_name}_trans.tra", sep = ' ')